# This is a notebook for querying the WB1 T&S instance for the 1.0.0 Beacon release.

-   You can run each cell individually by pressing "shift + enter".
-   For more information, questions, bugs, please contact us on Slack:
    -   https://join.slack.com/t/beacontechnic-wwa5548/shared_invite/zt-2dp1vv56r-tj_KFac0sAKNuAgUKPPDRg.


#### In order to get access to the Beacon endpoint, you need to fill in your unique personal token between the " " in the cell below.


In [1]:
Token = ""

#### Install the following packages, if you have not already installed them in your environment:

-   pip install requests
-   pip install xarray
-   pip install ipywidgets
-   pip install cartopy
-   pip install h5netcdf
-   pip install netcdf4
-   pip install scipy
-   pip install packaging


#### Import the required packages


In [2]:
import requests
import json
import xarray as xr
import datetime
import pandas as pd
import os
import cartopy.crs as ccrs
import cartopy.feature as cfeature
import matplotlib.pyplot as plt

#### Retrieve the available columns from the WB1 Merged endpoint

Swagger page here: https://beacon-wb1-ts.maris.nl/swagger/


In [3]:
responseinfo = requests.get(
    "https://beacon-wb1-ts.maris.nl/api/query/available-columns",
    headers={"Authorization": f"Bearer {Token}"},
)
params = responseinfo.json()

#### Below you can search through the available columns by entering text between the brackets of search_columns(" ").


In [4]:
def search_columns(search_term):
    search_term = search_term.lower()
    matches = [col for col in params if search_term in col.lower()]

    if matches:
        print("Matching columns:")
        for match in matches:
            print(match)
    else:
        print("No matching columns found.")


search_columns("COMMON")  # Enter your search term here

Matching columns:
COMMON_EDMO_CODE
COMMON_ODV_TAG
COMMON_TIME
COMMON_TIME_QC
COMMON_DEPTH
COMMON_DEPTH_P01
COMMON_DEPTH_P06
COMMON_DEPTH_STANDARD_NAME
COMMON_DEPTH_UNITS
COMMON_LONGITUDE
COMMON_LONGITUDE_STANDARD_NAME
COMMON_LONGITUDE_UNITS
COMMON_LONGITUDE_P01
COMMON_LONGITUDE_P06
COMMON_LATITUDE_STANDARD_NAME
COMMON_LATITUDE_UNITS
COMMON_LATITUDE
COMMON_LATITUDE_P01
COMMON_LATITUDE_P06
COMMON_POSITION_QC
COMMON_TEMPERATURE
COMMON_TEMPERATURE_QC
COMMON_TEMPERATURE_P01
COMMON_TEMPERATURE_P06
COMMON_TEMPERATURE_STANDARD_NAME
COMMON_TEMPERATURE_UNITS
COMMON_SALINITY
COMMON_SALINITY_QC
COMMON_SALINITY_P01
COMMON_SALINITY_P06
COMMON_SALINITY_STANDARD_NAME
COMMON_DEPTH_QC
COMMON_TEMPERATURE_L22
COMMON_SALINITY_L22
COMMON_SALINITY_UNITS
COMMON_TEMPERATURE_L05
COMMON_TEMPERATURE_L06
COMMON_TEMPERATURE_L35
COMMON_SALINITY_L05
COMMON_SALINITY_L06
COMMON_SALINITY_L35
COMMON_EDMERP_CODE
COMMON_CSR
COMMON_DATE_UPDATE
COMMON_PLATFORM_L06
COMMON_PLATFORM_B76
COMMON_PLATFORM_C17
COMMON_INSTRUMENT_L05

#### This will create the query body based on your input parameters, you can add other "query_parameters" and "filters" to suit your needs.

-   For more query examples and explanations, you can take a look at https://maris-development.github.io/beacon/.


In [11]:
query = {
    "query_parameters": [
        {
            "column_name": "COMMON_TEMPERATURE",
        },
        {
            "column_name": "COMMON_TEMPERATURE_QC",
        },
        {
            "column_name": "COMMON_TEMPERATURE_P01",
        },
        {
            "column_name": "COMMON_TEMPERATURE_P06",
        },
        {"column_name": "COMMON_TIME", "alias": "TIME"},
        {"column_name": "COMMON_TIME_QC", "alias": "TIME_QC"},
        {"column_name": "COMMON_DEPTH"},
        {"column_name": "COMMON_DEPTH_QC"},
        {"column_name": "COMMON_LONGITUDE"},
        {"column_name": "COMMON_LATITUDE"},
        {"column_name": "COMMON_POSITION_QC"},
        {"column_name": "COMMON_EDMO_CODE"},
        {"column_name": "SOURCE_BDI"},
        {"column_name": "SOURCE_BDI_DATASET_ID"},
    ],
    "filters": [
        {
            "for_query_parameter": "TIME",
            "min": "2003-01-01T00:00:00",
            "max": "2003-12-01T00:00:00",
        },
        {"for_query_parameter": "COMMON_LONGITUDE", "min": -8, "max": 37},
        {"for_query_parameter": "COMMON_LATITUDE", "min": 30, "max": 46},
        {
            "is_not_null": {"for_query_parameter": "COMMON_TEMPERATURE"},
        },
    ],
    "output": {"format": "ipc"},
}

#### This is the post request that is sent to Beacon with the above specified body.


In [12]:
response = requests.post(
    "https://beacon-wb1-ts.maris.nl/api/query",
    json.dumps(query),
    headers={"Authorization": f"Bearer {Token}",
             "Content-type": "application/json"},
)

if response.status_code == 204:
    print(
        "No data has been found for your query, please update your input fields above and run the notebook again."
    )
elif response.status_code != 200:
    print(response.text)

In [25]:
if not os.path.exists("./Beacon_V1.0.0_Output"):
    os.makedirs("Beacon_V1.0.0_Output")

open(
    f"./Beacon_V1.0.0_Output/WB1-Merged.arrow",
    "wb",
).write(response.content)
df = pd.read_feather(f"./Beacon_V1.0.0_Output/WB1-Merged.arrow")
df = df.set_index("TIME").sort_index()

SOURCE_BDI
BEACON-SEADATANET    3761
BEACON-WOD           1897
BEACON-CORA-TS       1053
BEACON-CORA-PR        776
BEACON-ARGO           281
Name: count, dtype: int64

In [8]:
import xarray as xr

xr = df.to_xarray()
xr.to_netcdf(f"./Beacon_V1.0.0_Output/WB1-Merged.nc")